In [1]:
import sys, os, time, shutil, random
from pathlib import Path
_cwd = os.getcwd()
os.chdir(Path(_cwd)/'..')
from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:98% !important; }</style>"))
# %load_ext autoreload
# %autoreload 2
# %pdb
import h5py as hf
import omnifig as fig
import numpy as np
np.set_printoptions(linewidth=120)
import pickle
from tabulate import tabulate
from tqdm import tqdm_notebook as tqdm
import torch
from torch import nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
# import torchvision.models
from torch.utils.data import Dataset, DataLoader, TensorDataset
# import timm

%matplotlib notebook
import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.cm
from matplotlib import animation
import matplotlib as mpl
# mpl.rc('image', cmap='gray')
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import normalize

fig.initialize()

import omnilearn as learn
from omnilearn import models
from omnilearn import util
from omnilearn.data import InterventionSamplerBase

from sklearn.decomposition import PCA
import sklearn.datasets

# from src import sample_full_interventions, response_mat, factor_reponses

# import gpumap
from c3linearize import linearize, class_graph


2021-11-11 18:05:07.585592: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /is/ei/fleeb/miniconda3/lib/python3.8/site-packages/cv2/../../lib64:/is/ei/fleeb/.mujoco/mujoco200/bin
2021-11-11 18:05:07.585608: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
base_props = {'ID': 'inception-v3', 'pretrained': True}

In [11]:
dataset = fig.quick_run('load-data', 'd/mnist')
len(dataset)

| dataset: [<class 'omnifig.config.ConfigDict'>]
| CREATING . (type=dataset)
|  > .num_workers: 0
|  > batch_size: 128
|  > shuffle: True
|  > drop_last: True
|  > .step_device --> .device: 'cuda'
|  > infinite: False (by default)
|  > extractor: None (by default)
|  > allow_batched: True (by default)
|  > split [dict with 1 item/s]
|  >  > (val): 0.166666
|  > shuffle-split: True (by default)
|  > split-src: 'train' (by default)
|  > wrappers: [] (by default)
|  > default-mode --> mode: 'train' (by default)
|  > _dataset_type --> dataset-name --> name: 'mnist'
|  > _dataset_mod: [] (by default)
|  > mode-aliases [dict with 1 item/s]
|  >  > (eval): 'val'
|  > .seed: 0
|  > device: 'cpu'
|  > pin_memory: True (by default)
|  > epoch_seed --> .seed: 0
|  > skip_load: False (by default)
|  > CREATING . (type=dataset/mnist)
|  >  > dataroot: None (by default)
|  >  > mode: 'train'
|  >  > download: False
|  >  > train: True (by default)
|  >  > resize: True (by default)
|  >  > supervised

/is/ei/fleeb/miniconda3/lib/python3.8/site-packages/torch/nn/functional.py:3454: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


50000

In [12]:
root = dataset.get_root()
root

PosixPath('/is/ei/fleeb/workspace/local_data/mnist')

In [13]:
# name = f'{dataset.cat}_fid_stats.h5'
name = f'fid_stats.h5'

f = hf.File(root/name)
list(f.keys())

['train_192_mu',
 'train_192_sigma',
 'train_2048_mu',
 'train_2048_sigma',
 'train_64_mu',
 'train_64_sigma',
 'train_768_mu',
 'train_768_sigma']

In [14]:
# fid = fig.quick_create('loss', ident='frechet-distance')
# fid

In [15]:
# m1, s1 = torch.from_numpy(f['train_2048_mu'][:]), torch.from_numpy(f['train_2048_sigma'][:])
# m2, s2 = torch.from_numpy(f['test_2048_mu'][:]), torch.from_numpy(f['test_2048_sigma'][:])

# fid((m1, s1), (m2, s2))

In [16]:
for key in [key for key in f.keys() if key.endswith('_mu')]:
    mode, dim, _ = key.split('_')
    dim = int(dim)
    props = {'dim':dim, 'mode':mode, **base_props}
    
    dataset.save_stats(props, (torch.from_numpy(f[f'{mode}_{dim}_mu'][:]), 
                               torch.from_numpy(f[f'{mode}_{dim}_sigma'][:])))
    print(props)

{'dim': 192, 'mode': 'train', 'ID': 'inception-v3', 'pretrained': True}
{'dim': 2048, 'mode': 'train', 'ID': 'inception-v3', 'pretrained': True}
{'dim': 64, 'mode': 'train', 'ID': 'inception-v3', 'pretrained': True}
{'dim': 768, 'mode': 'train', 'ID': 'inception-v3', 'pretrained': True}


In [17]:
table = dataset.get_datafile('stats/table')
table

{'train-inception-v3': [[{'dim': 192,
    'mode': 'train',
    'ID': 'inception-v3',
    'pretrained': True},
   'train-inception-v3-0'],
  [{'dim': 2048, 'mode': 'train', 'ID': 'inception-v3', 'pretrained': True},
   'train-inception-v3-1'],
  [{'dim': 64, 'mode': 'train', 'ID': 'inception-v3', 'pretrained': True},
   'train-inception-v3-2'],
  [{'dim': 768, 'mode': 'train', 'ID': 'inception-v3', 'pretrained': True},
   'train-inception-v3-3']]}